## Import the libraries

In [27]:
from glove import Corpus, Glove
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import datetime,time
from flashtext.keyword import KeywordProcessor
from nltk.stem import WordNetLemmatizer
from string import punctuation
import re
import pandas as pd
import numpy as np
import string
import multiprocessing
from nltk.corpus import stopwords
from flashtext.keyword import KeywordProcessor
from sklearn.model_selection import train_test_split
from sklearn import metrics

# libraries for dataset preparation, feature engineering, model training 
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
#from utils.language_models import WordVectorEmbedding
from gensim.test.utils import datapath, get_tmpfile
#import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.utils import to_categorical
from sklearn import preprocessing
from pymongo import MongoClient

# Import Translator module from googletrans package
from googletrans import Translator  

## Connect to MongoDB to get the twitter crawler dataset containing the streamed #election2019 related tweets

In [4]:
MONGO_HOST = 'localhost'
MONGO_PORT = 27017
DATABASE_NAME='Portfolio'

In [5]:
client_prod = MongoClient(host=MONGO_HOST,port=MONGO_PORT,unicode_decode_error_handler='ignore')
db_prod = client_prod[DATABASE_NAME]
collection_name = "Election_2019"
collection=db_prod[collection_name]
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, unicode_decode_error_handler='ignore'), 'Portfolio'), 'Election_2019')

In [6]:
df_tweets=pd.DataFrame(list(collection.find({})))
print('Total number of tweets loaded from Mongo : {}'.format(df_tweets.shape[0]))
df_tweets.head()

Total number of tweets loaded from Mongo : 6871


,Created At,Likes,Location,Name,Retweet,Tweet,Tweet_cleaned,_id
0,2019-03-24 01:45:07,0,India,Amar Ujala,0,https://t.co/5n9MTfwWtw\n@BJP4India @INCIndia ...,https electioncommission,5c96e12a5c284ca44cf2d1d9
1,2019-03-24 01:45:22,0,"Ohio, USA",JP,0,RT @ThanthiTV: திமுக கூட்டத்தில் தூக்கம்... மத...,rt https dmk,5c96e1395c284ca44cf2d1da
2,2019-03-24 01:45:54,0,None,BenarNews Thai,0,พลเอกประยุทธ์ จันทร์โอชา หย่อนบัตรลงคะแนน ที่เ...,bangkok thailand Benarnewsthai https,5c96e15a5c284ca44cf2d1db
3,2019-03-24 01:46:11,0,"Tirunelveli, India",கோபிராஜ்.க - GopiRaj.K,0,RT @ThanthiTV: திமுக கூட்டத்தில் தூக்கம்... மத...,rt https dmk,5c96e16b5c284ca44cf2d1dc
4,2019-03-24 01:46:25,0,"New Delhi, Delhi",Paresh Kumar,0,"Common Indian voter: हे वराह पुत्र, अंजुलीभर ज...",common indian voter corrupt politicians corrup...,5c96e1795c284ca44cf2d1dd


## As we can see, tweets were generated in various languages. 
#### I didn't cap them to any specific one at the source since that would have resulted in a heavily biased data towards the only english speaking Indian community. I have previously built a language classifier for Germany and English text detection, however since we will be dealing with a vast number of languages, we would stick to google's own translator API

In [28]:
# Language Translator
translator = Translator() # Create object of Translator.

In [8]:
translated = translator.translate(df_tweets.Tweet[3]) 
translated.text

'RT @ThanthiTV: Sleep at DMK meeting ... alcoholism ...\nhttps://t.co/1U9JBZmvsS\n#DMK | # Election2019'

In [29]:
def translate_tweet(tweet):
    try:
        translated = translator.translate(tweet) 
    except:
        translated=tweet
        return tweet
    return translated.text

In [11]:
df_tweets['Tweet'][0:25]

0     https://t.co/5n9MTfwWtw\n@BJP4India @INCIndia ...
1     RT @ThanthiTV: திமுக கூட்டத்தில் தூக்கம்... மத...
2     พลเอกประยุทธ์ จันทร์โอชา หย่อนบัตรลงคะแนน ที่เ...
3     RT @ThanthiTV: திமுக கூட்டத்தில் தூக்கம்... மத...
4     Common Indian voter: हे वराह पुत्र, अंजुलीभर ज...
5     RT @AmarUjalaNews: https://t.co/5n9MTfwWtw\n@B...
6     RT @ThanthiTV: திமுக கூட்டத்தில் தூக்கம்... மத...
7     RT @JustDoGood3: This is part of the answer! 👇...
8     RT @ThanthiTV: திமுக கூட்டத்தில் தூக்கம்... மத...
9     RT @AmarUjalaNews: https://t.co/5n9MTfwWtw\n@B...
10    RT @EasternHerald: JKLF banning on Pakistan da...
11    RT @ThanthiTV: நாடாளுமன்ற தேர்தல் - நாம் தமிழர...
12    RT @Matcha_Phornin: It’s happening now, #Thail...
13    RT @nakkheeranweb: தி.மு.க.வின் கனிமொழிக்காக வ...
14    3 in 1 สุดๆ #SanookNews #ข่าววันนี้ #เลือกตั้ง...
15    RT @MakkalSelvarTTV: அம்மா மக்கள் முன்னேற்றக் ...
16    RT @johnlittle: @deniseshrivell A disappointin...
17    RT @ThanthiTV: திமுக கூட்டத்தில் தூக்கம்..

In [25]:
#Temp
text='RT @john3little: @denise32shrivell A disappointing attack https://www.google.com'
text=re.sub(r"RT @[\w]*:",'',text)
text=re.sub(r"http\S+", "", text)
text=re.sub(r"([^a-zA-Z0-9#])+",' ',text)
text=re.sub(r"\s\s+",' ',text)
text=text.strip()
text

'denise32shrivell A disappointing attack'

In [34]:
st='RT @arulgreen: #விகடன் சர்வே: 21 வயதுக்கு உடபட்டோரின் திருமணத்துக்கு பெற்றோர் ஒப்புதல் தேவை - #பாமகவின் கோரிக்கைக்கு தமிழக மக்கள் மாபெரும்'
stt=translate_tweet(st)
#removing all retweers
text=re.sub(r"RT @[\w]*:",'',stt)
            
text=re.sub(r"http\S+", "", text)
text=re.sub(r"([^a-zA-Z0-9#])+",' ',text)
text=re.sub(r"\s\s+",' ',text)
text=text.strip()
text

'# 21 #'

In [11]:
df_tweets_translated=df_tweets['Tweet'].apply(translate_tweet)
df_tweets_translated[0:50]

0     https://t.co/5n9MTfwWtw\n@ BJP4India @INCIndia...
1     RT @ThanthiTV: Sleep at DMK meeting ... alcoho...
2     General Prayut Chan-ocha dropped the ballot. A...
3     RT @ThanthiTV: Sleep at DMK meeting ... alcoho...
4     Common Indian voter: O son of the son, be subm...
5     RT @AmarUjalaNews: https://t.co/5n9MTfwWtw\n@ ...
6     RT @ThanthiTV: Sleep at DMK meeting ... alcoho...
7     RT @JustDoGood3: This is part of the answer! 👇...
8     RT @ThanthiTV: Sleep at DMK meeting ... alcoho...
9     RT @AmarUjalaNews: https://t.co/5n9MTfwWtw\n@ ...
10    RT @EasternHerald: JKLF banning on Pakistan da...
11    RT @ThanthiTV: Parliamentary Elections - We Se...
12    RT @Matcha_Phornin: It’s happening now, #Thail...
13    RT @nakkheeranweb: தி.மு.க.வின் கனிமொழிக்காக வ...
14    3 in 1 สุดๆ #SanookNews #ข่าววันนี้ #เลือกตั้ง...
15    RT @MakkalSelvarTTV: அம்மா மக்கள் முன்னேற்றக் ...
16    RT @johnlittle: @deniseshrivell A disappointin...
17    RT @ThanthiTV: Sleep at DMK meeting ... al

In [12]:
df_tweets['Tweet_translated']=df_tweets_translated

In [13]:
df_tweets

,Created At,Likes,Location,Name,Retweet,Tweet,Tweet_translated
0,2019-03-24 01:45:07,0,India,Amar Ujala,0,https://t.co/5n9MTfwWtw\n@BJP4India @INCIndia ...,https://t.co/5n9MTfwWtw\n@ BJP4India @INCIndia...
1,2019-03-24 01:45:22,0,"Ohio, USA",JP,0,RT @ThanthiTV: திமுக கூட்டத்தில் தூக்கம்... மத...,RT @ThanthiTV: Sleep at DMK meeting ... alcoho...
2,2019-03-24 01:45:54,0,None,BenarNews Thai,0,พลเอกประยุทธ์ จันทร์โอชา หย่อนบัตรลงคะแนน ที่เ...,General Prayut Chan-ocha dropped the ballot. A...
3,2019-03-24 01:46:11,0,"Tirunelveli, India",கோபிராஜ்.க - GopiRaj.K,0,RT @ThanthiTV: திமுக கூட்டத்தில் தூக்கம்... மத...,RT @ThanthiTV: Sleep at DMK meeting ... alcoho...
4,2019-03-24 01:46:25,0,"New Delhi, Delhi",Paresh Kumar,0,"Common Indian voter: हे वराह पुत्र, अंजुलीभर ज...","Common Indian voter: O son of the son, be subm..."
5,2019-03-24 01:46:31,0,"Patna City Outer, India",🇮🇳 Lucky Kumar 💓,0,RT @AmarUjalaNews: https://t.co/5n9MTfwWtw\n@B...,RT @AmarUjalaNews: https://t.co/5n9MTfwWtw\n@ ...
6,2019-03-24 01:46:33,0,chennai,senthil kumar🇮🇳,0,RT @ThanthiTV: திமுக கூட்டத்தில் தூக்கம்... மத...,RT @ThanthiTV: Sleep at DMK meeting ... alcoho...
7,2019-03-24 01:46:37,0,On Bunnerong land,sue travers,0,RT @JustDoGood3: This is part of the answer! 👇...,RT @JustDoGood3: This is part of the answer! 👇...
8,2019-03-24 01:47:20,0,None,Nirmal 🔦,0,RT @ThanthiTV: திமுக கூட்டத்தில் தூக்கம்... மத...,RT @ThanthiTV: Sleep at DMK meeting ... alcoho...
9,2019-03-24 01:47:26,0,"Agra, India",B M Singh Dr 1 chowkidar,0,RT @AmarUjalaNews: https://t.co/5n9MTfwWtw\n@B...,RT @AmarUjalaNews: https://t.co/5n9MTfwWtw\n@ ...


In [18]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)        
    return input_txt
def clean_tweets(lst):
    # remove twitter Return handles (RT @xxx:)
    lst = np.vectorize(remove_pattern)(lst, "RT @[\w]*:")
    # remove twitter handles (@xxx)
    lst = np.vectorize(remove_pattern)(lst, "@[\w]*")
    # remove URL links (httpxxx)
    lst = np.vectorize(remove_pattern)(lst, "https?://[A-Za-z0-9./]*")
    # remove special characters, numbers, punctuations (except for #)
    lst = np.core.defchararray.replace(lst, "[^a-zA-Z#]", " ")
    return lst

In [27]:
clean_tweets=df_tweets['Tweet_translated'].apply(clean_tweets)
clean_tweets

0       \n BJP4India INCIndia #ElectionCommission # El...
1        Sleep at DMK meeting ... alcoholism ...\n\n#D...
2       General Prayut Chan-ocha dropped the ballot. A...
3        Sleep at DMK meeting ... alcoholism ...\n\n#D...
4       Common Indian voter: O son of the son, be subm...
5        \n BJP4India INCIndia #ElectionCommission # E...
6        Sleep at DMK meeting ... alcoholism ...\n\n#D...
7        This is part of the answer! 👇👇\n\nPrinting of...
8        Sleep at DMK meeting ... alcoholism ...\n\n#D...
9        \n BJP4India INCIndia #ElectionCommission # E...
10       JKLF banning on Pakistan day right before #Lo...
11       Parliamentary Elections - We Seeman published...
12       It’s happening now, #Thailand #Election2019 ,...
13       தி.மு.க.வின் கனிமொழிக்காக வாக்கு சேகரிப்பது எ...
14      3 in 1 สุดๆ #SanookNews #ข่าววันนี้ #เลือกตั้ง...
15       அம்மா மக்கள் முன்னேற்றக் கழக ஆதரவு பெற்ற SDPI...
16        A disappointing result for #Labour but its o...
17       Sleep

In [30]:
df_tweets['Tweet_cleaned']=clean_tweets
df_tweets.head()

,Created At,Likes,Location,Name,Retweet,Tweet,Tweet_translated,Tweet_cleaned
0,2019-03-24 01:45:07,0,India,Amar Ujala,0,https://t.co/5n9MTfwWtw\n@BJP4India @INCIndia ...,https://t.co/5n9MTfwWtw\n@ BJP4India @INCIndia...,\n BJP4India INCIndia #ElectionCommission # El...
1,2019-03-24 01:45:22,0,"Ohio, USA",JP,0,RT @ThanthiTV: திமுக கூட்டத்தில் தூக்கம்... மத...,RT @ThanthiTV: Sleep at DMK meeting ... alcoho...,Sleep at DMK meeting ... alcoholism ...\n\n#D...
2,2019-03-24 01:45:54,0,None,BenarNews Thai,0,พลเอกประยุทธ์ จันทร์โอชา หย่อนบัตรลงคะแนน ที่เ...,General Prayut Chan-ocha dropped the ballot. A...,General Prayut Chan-ocha dropped the ballot. A...
3,2019-03-24 01:46:11,0,"Tirunelveli, India",கோபிராஜ்.க - GopiRaj.K,0,RT @ThanthiTV: திமுக கூட்டத்தில் தூக்கம்... மத...,RT @ThanthiTV: Sleep at DMK meeting ... alcoho...,Sleep at DMK meeting ... alcoholism ...\n\n#D...
4,2019-03-24 01:46:25,0,"New Delhi, Delhi",Paresh Kumar,0,"Common Indian voter: हे वराह पुत्र, अंजुलीभर ज...","Common Indian voter: O son of the son, be subm...","Common Indian voter: O son of the son, be subm..."
